## E-careers.com

In [1]:
##Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
from datetime import date

In [2]:
# Get all the category links
cat_link = [
    'https://www.e-careers.com/courses/accounting',
    'https://www.e-careers.com/courses/bookkeeping',
    'https://www.e-careers.com/courses/business-management',
    'https://www.e-careers.com/courses/cemap',
    'https://www.e-careers.com/courses/childcare',
    'https://www.e-careers.com/courses/coding-web-development',
    'https://www.e-careers.com/courses/cyber-security',
    'https://www.e-careers.com/courses/digital-marketing',
    'https://www.e-careers.com/courses/football',
    'https://www.e-careers.com/courses/microsoft-office',
    'https://www.e-careers.com/courses/networking-it',
    'https://www.e-careers.com/courses/project-management',
    'https://www.e-careers.com/courses/teaching',
    'https://www.e-careers.com/courses/workplace-safety'
           ]

In [3]:
# Create a function, whose input is category link
def scrape(category_link):
    title = []
    price = []
    link = []
    
    for pg in range(1,5):
        r = requests.get(category_link+f'?page={pg}#courseListing')
        s = BeautifulSoup(r.content,'html.parser')
        
        # Main container
        for res in s.find_all('div',class_='single-course-grid'):

            #Title
            try:
                title.append(res.find('div',class_='single-course-name').text)
            except:
                title.append('missing')

            # Price
            try:
                price.append(res.find('span',class_='now_price').text)
            except:
                price.append('missing')

            link.append(res.a.get('href'))

    # Create a df with the extracted vars
    df = pd.DataFrame({'title':title,
                      'link':link,
                      'price':price})
    # Extract category
    df['cat'] = category_link.split('/')[4]
    return df.to_csv(f'{df.cat.iloc[0]}.csv',index=False)

In [5]:
# Iterate all the sub category links
for lnk in cat_link:
    scrape(lnk)

# Create a csv file by merging all the csv files
today = date.today().strftime('%d_%b')
df = pd.concat([pd.read_csv(f, encoding='latin1') for f in glob.glob('*.csv')],sort=False).reset_index(drop=True)
df.to_csv(f'{today}_ecareers.csv',index=False)